# CNN with Keras by TensorFlow

# 1.0 Dependencies and Notes

This notebook was built with the libraries imported below and the following versions:

Pandas 2.2.3 <br>
Numpy 2.0.2 <br>
Altair 5.4.1 <br>
sklearn 1.5.0 <br>
Keras 3.6.0 <br>

Different versions of these libraries may affect the functionality of this notebook.

The purpose of this notebook is to create a convolutional neural network to predict remaining useful life of jet engines using data provided by NASA. The notebook includes definitions to build the model, fit it, and then explore and store the results. 

Results are stored via a CSV file. There is a function for looping through different parameters, and other functions for viewing, exploring, and saving the results. 

In [121]:
import pandas as pd
import numpy as np
import altair as alt
import sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from numpy import array, hstack
import pickle
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from keras import Input

In [124]:
print(pd.__version__)
print(np.__version__)
print(alt.__version__)
print(sklearn.__version__)
print(keras.__version__)

2.2.3
2.0.2
5.4.1
1.5.0
3.6.0


## 1.1 Load and define train and test data. 

In [81]:
#link = 'processed_data_pickle_files_no_smoothing/'
link = 'processed_data_pickle_files_ref/'

with open(link + 'test_data_batches.pkl', 'rb') as file:
    test = pickle.load(file)
    
with open(link + 'train_data_batches.pkl', 'rb') as file:
    train = pickle.load(file)    
    
with open(link + 'train_target_values.pkl', 'rb') as file:
    y_train = pickle.load(file)
    
with open(link + 'true_rul_values.pkl', 'rb') as file:
    y_test = pickle.load(file)

In [82]:
display(train.shape)
display(y_train.shape)
display(test.shape)
display(y_test.shape)

(17731, 30, 14)

(17731,)

(100, 30, 14)

(100,)

# 2.0 Definitions

In [83]:
def make_cnn(params = default_params, print_summary = False):
    
    """
    Intializes a convolutional neural network using TensorFlow Keras Library. 
    Parmeters are entered as a library. 
    
    Inputs:
    params: library of input parameters, must include 'sec_conv', '#filters_conv1', 'filter_size_conv1', 
            'act_func_conv1', 'input_1', and 'input_2'; may need to include others. 
    print_summary: boolean that dictates whether to print a summary of the constructed model 
            (provides number of nuerons in each network, etc.)
    """
    
    
    #A sequential model can have different NN layers added. Use a list to define layers, or use add method.
    #Sequential models only work for one input tensor and one output tensor (including for each layer).
    model = Sequential()
    
    #Input(shape = (params['input_1'], params["input_2"]))
    
    #Adds a 1-dimensional convolution layer (only moves in one direction: down each 'sample' of timeseries).
    model.add(Conv1D(filters = params["#filters_conv1"], #dimension of output space (number of filters)
                     kernel_size = params["filter_size_conv1"], #size of the convolution window
                     strides = 1, #convolution layer stride length
                     padding = 'valid', #zero padding at ends of convolutions, options: 'valid' (no padding), 'same'
                     activation=params["act_func_conv1"], #activation function to derive non-linear relationships; default is None.
                     input_shape=(params['input_1'], params["input_2"]) #steps followed by number of features (same order as input)
                    )) 
    
    #Adds a second 1D convolution filter if user requests.
    if params["sec_conv"] == True:
        model.add(Conv1D(filters = params["#filters_conv2"], #dimension of output space (number of filters)
                     kernel_size = params["filter_size_conv2"], #size of the convolution window
                     activation=params["act_func_conv2"], #activation function to derive non-linear relationships; default is None.
                        )) 
    
    #Adds a pooling layer to reduce dimensions/computation time - Max or Average is available. 
    #Downsizes the input from the Conv1D layer, so structure is kept but some info is lost.
    model.add(MaxPooling1D(pool_size = 2)) #Number of features considered at once.
    
    #Adds a flattening layer, which makes the 3D data into a 1D array so it's compatible with Dense Layers.
    model.add(Flatten())
    
    #Adds a "dense" layer, which is a regular NN layer. This interprets the output of the previous layers.
    model.add(Dense(50, #First parameter is the output space dimensionality.
                    activation = 'relu')) 
    
    #Adds a second dense layer to reduce the 50 neurons to a single output.
    #This is done seperately for each batch.
    model.add(Dense(1))
    
    #This method configures the model for training. It's where you choose parameters.
    #Other parameter available: loss_weights, metrics, weighted_metrics, etc. 
    model.compile(optimizer = 'adam', loss = 'mse', metrics = ["mean_squared_error", 
                                                               "root_mean_squared_error", 
                                                               "mean_absolute_error", 
                                                               "mean_absolute_percentage_error", 
                                                               "r2_score"])
    
    if print_summary == True:
        model.summary()
    
    return model

In [84]:
def cnn_run_for_log(model, train_data, train_target, test_data, test_target, params = default_params):
    
    """ 
    Trains an input model on the input train data, then collects various scoring metrics of both the 
    train and test data. The input parameters dictionary is then concatenated with the metrics to provide 
    a dictionary of both the metrics and input parameters used. 
    
    Inputs:
    
    Model: CNN model from the previous function above ('make_cnn') or CNN defined via other means
    
    Various data inputs: Train and Test, plus targets
    
    params: library of parameters. Must include 'perform_validation' and '#_epochs'.
    
    Output: library of train and test parameters, along with parameters included in the 'params' input. 
    
    """
    
    if params['perform_validation'] == True:
        v_s = 0.2
    else:
        v_s = None
    
    history = model.fit(train_data, 
                      train_target, 
                      validation_split = v_s, 
                      epochs = params["#_epochs"], 
                      verbose = 0
                     )
    logger = {}
    
    y_hat = model.predict(test_data, verbose = 0)
    
    logger["train_MSE"] = history.history["mean_squared_error"][-1]
    logger["test_MSE"] = mean_squared_error(test_target, y_hat).item()
    
    logger["train_RMSE"] = history.history["root_mean_squared_error"][-1]
    logger["test_RMSE"] = np.sqrt(mean_squared_error(test_target, y_hat)).item()
    
    logger["train_MAE"] = history.history["mean_absolute_error"][-1]
    logger["test_MAE"] = mean_absolute_error(test_target, y_hat).item()
    
    logger["train_MAPE"] = history.history["mean_absolute_percentage_error"][-1]
    logger["test_MAPE"] = np.mean(np.abs((test_target - y_hat) / y_test)).item() * 100
    
    logger["train_R2"] = history.history["r2_score"][-1]
    logger["test_R2"] = r2_score(test_target, y_hat)
    
    logger.update(params)
    
    return logger
    
    

In [85]:
def add_to_logger(new_instance, existing_dict):
    
    """
    Takes in a new instance of the function 'run_cnn_for_log' which returns the performance metrics 
    for a CNN using the listed input parameters. It then adds that instance to a dictionary of lists,
    where each index in the list represents a new run of 'run_cnn_for_log'. This is intended to be used
    in running loops during parameter tuning to keep track of which parameters perform the best. 
    
    Input:
    new_instance: a dictionary of the most recent parameters and performance metrics
    existing_dict: a dictionary of lists that keep a record of performance metrics and the parameters that
            led to those results
    
    Output:
    An updated record of parameters/performance metrics in which the most recent parameters are added to the record
    
    """
    
    if existing_dict == None:
        record = {}
        for key in new_instance.keys():
            record[key] = []
    else:
        record = existing_dict.copy()
        
    for key in record.keys():
        l = record[key]
        if key in new_instance.keys():
            l.append(new_instance[key])     
        else:
            l.append(np.nan)
        record[key] = l
        
    return record

In [89]:
def loop_through_parameters(loops, 
                            params = default_params, 
                            train = train, 
                            y_train = y_train, 
                            test = test, 
                            y_test = y_test):
    """
    Runs parameter loops for model and records the resulting metrics. Returns a dictionary that is 
    a log of the results, where each key represents a parameter or performance metric, and each item is
    a list where the indexes represent the runs in chronological order. 
    
    Inputs:
    loops: a dictionary of the the keys and values to loop through.
    params: a dictionary with all the parameters neccessary to build the CNN, train it, and acquire the 
    results using the functions defined previously.
    
    Output:
    A dictionary of lists with input parameters and resultant performance metrics. Can easily be used
    to construct a dataframe. 
    """
    
    record = None
    keys = list(loops.keys())
    
    def nested_function(p1 = None, p2 = None, p3 = None, p4 = None):
        params[keys[0]] = p1
        if len(keys) > 1:
            params[keys[1]] = p2
        if len(keys) > 2:
            params[keys[2]] = p3
        if len(keys) > 3:
            params[keys[3]] = p4
            
        model = make_cnn(params = params)
        new_instance = cnn_run_for_log(model, train, y_train, test, y_test, params = params)
        r = add_to_logger(new_instance, record)
        return r
        
    if len(loops.keys()) == 1:
        for p_1 in loops[keys[0]]:
            record = nested_function(p1 = p_1)
            
    elif len(loops.keys()) == 2:
        for p_1 in loops[keys[0]]:
            for p_2 in loops[keys[1]]:
                record = nested_function(p1 = p_1, p2 = p_2)
                
    elif len(loops.keys()) == 3:
        for p_1 in loops[keys[0]]:
            for p_2 in loops[keys[1]]:
                for p_3 in loops[keys[2]]:
                    record = nested_function(p1 = p_1, p2 = p_2, p3 = p_3)
                    
    else:
        for p_1 in loops[keys[0]]:
            for p_2 in loops[keys[1]]:
                for p_3 in loops[keys[2]]:
                    for p_4 in loops[keys[3]]:
                        record = nested_function(p1 = p_1, p2 = p_2, p3 = p_3, p4 = p_4)
                        
    return record
        

In [ ]:
def add_to_cnn_log(record, link = 'CNN_log.csv', save_changes = False):
    
    """
    Combines most recent group of models with those saved in the log file. Includes designating a
    tuning group based on the most recent tuning group in the log file.
    
    Inputs:
    record: most recent record set of parameter tunings, as returned by above functions.
    link: pathway and filename for the saved log file, it if exists. If it doesn't exist, this 
        function won't work.
    save_changes: designates whether to save the updates to the CSV file that stores the model parameter tuning results.
    
    Output:
    A dataframe of the combines records on file and the most recent turning group. 
    """
    
    df = pd.read_csv(link)
    group_num = df['tuning_group'].max() + 1
    r = pd.DataFrame(record)
    r.insert(0, 'tuning_group', group_num)
    
    df = pd.concat([df, r], ignore_index = True)

    if save_changes == True:
        df.to_csv(link, index = False)
    
    return df

In [111]:
def plot_results(data, fields = list(loops.keys())):
    
    """
    Makes a simple Altair Chart for compairing results visually.
    
    Input:
    data: Dataframe that includes the fields from the most current record or from the CNN_log saved as a CSV.
    fields: designated fields to encode using shape and color. Default are the first two designated in the 
        most recent record.
        
    Output:
    A chart comparing changes in the designated fields, mapped against test RMSE and difference between
        train and test RMSE.
    """

    data['RMSE_diff'] = data['test_RMSE'] - data['train_RMSE']
    fields_to_keep = ["test_RMSE", "train_RMSE", "RMSE_diff"] + fields
    data = data[fields_to_keep]

    chart = alt.Chart(data).mark_point().encode(x = "test_RMSE", 
                                                y = 'RMSE_diff', 
                                                color = fields[0] + ":N", 
                                                shape = fields[1] + ":N")

    return chart

# 3.0 Building Models and Exploring Results

## Define the default parameters and those to change while looping. 

In [112]:
#Parameter List
default_params = {"sec_conv": False, 
          "perform_validation": False,
          "num_train_samples": train.shape[0],
          "input_1": train.shape[1], 
          "input_2": train.shape[2],
          "#_epochs": 5,
          "#filters_conv1": 32,
          "filter_size_conv1": 3,
          "act_func_conv1": 'sigmoid',
          "#filters_conv2": 16,
          "filter_size_conv2": 5,
          "act_func_conv2": 'relu',
         }

loops = {
         "#_epochs": [3, 5, 7, 9, 15], 
         "#filters_conv1": [32, 64, 128, 256, 512],
        }

In [113]:
#for each key and associated list in 'loops', make a record of results for different parameters.
record = loop_through_parameters(loops)

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Compare results graphically and in a dataframe

In [125]:
plot_results(pd.DataFrame(record))

alt.Chart(...)

In [126]:
df = pd.DataFrame(record)
print(df.columns)

##USE CTRL + "/" TO COMMENT OUT FIELDS
df = df[[
#     'train_MSE', 
#     'test_MSE', 
    'train_RMSE', 
    'test_RMSE', 
#     'train_MAE',
#     'test_MAE', 
#     'train_MAPE', 
#     'test_MAPE', 
#     'train_R2', 
#     'test_R2',
#     'sec_conv', 
#     'perform_validation', 
#     'num_train_samples', 
#     'input_1',
#     'input_2', 
    '#_epochs', 
    '#filters_conv1', 
#     'filter_size_conv1',
#     'act_func_conv1', 
#     '#filters_conv2', 
#     'filter_size_conv2',
#     'act_func_conv2', 
    ]]
df.insert(2, "RMSE_diff", df["test_RMSE"] - df["train_RMSE"])
display(df)

Index(['train_MSE', 'test_MSE', 'train_RMSE', 'test_RMSE', 'train_MAE',
       'test_MAE', 'train_MAPE', 'test_MAPE', 'train_R2', 'test_R2',
       'sec_conv', 'perform_validation', 'num_train_samples', 'input_1',
       'input_2', '#_epochs', '#filters_conv1', 'filter_size_conv1',
       'act_func_conv1', '#filters_conv2', 'filter_size_conv2',
       'act_func_conv2'],
      dtype='object')


,train_RMSE,test_RMSE,RMSE_diff,#_epochs,#filters_conv1
0,16.865767,16.746578,-0.119188,3,32
1,16.465771,15.957988,-0.507783,3,64
2,15.388669,15.314123,-0.074546,3,128
3,14.479988,15.629978,1.149990,3,256
4,14.377477,17.682569,3.305092,3,512
5,14.046468,15.673271,1.626803,5,32
6,14.068240,15.615329,1.547089,5,64
7,13.978996,15.345747,1.366751,5,128
8,13.692548,14.984304,1.291756,5,256
9,13.385993,15.564890,2.178897,5,512


In [16]:
log = None

for num_filters2 in [16, 32, 64, 128]:
    for num_filters1 in [32, 48, 64, 128]:
        for filter_size2 in [2, 3, 5, 8]:
            for sec_fil in [True]:
                temp_params = default_params.copy()
                temp_params["#filters_conv2"] = num_filters2
                temp_params["#filters_conv1"] = num_filters1
                temp_params["filter_size_conv2"] = filter_size2
                #temp_params['act_func_conv1'] = activation_function
                #temp_params['perform_validation'] = val
                temp_params['sec_conv'] = sec_fil
                cnn = make_cnn(params = temp_params, print_summary = False)
                inst = cnn_run_for_log(cnn, train, y_train, test, y_test, params = temp_params)
                if log == None:
                    log = {}
                    for k in inst.keys():
                        log[k] = []
                log = add_to_logger(inst, log)

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

In [401]:
pd.get_option('display.max_rows')

60

In [17]:
log.keys()

dict_keys(['train_MSE', 'test_MSE', 'train_RMSE', 'test_RMSE', 'train_MAE', 'test_MAE', 'train_MAPE', 'test_MAPE', 'train_R2', 'test_R2', 'sec_conv', 'perform_validation', 'num_train_samples', 'input_1', 'input_2', '#_epochs', '#filters_conv1', 'filter_size_conv1', 'act_func_conv1', '#filters_conv2', 'filter_size_conv2', 'act_func_conv2'])

In [18]:
pd.set_option('display.max_rows', None)
df = pd.DataFrame(log)
new = df['test_RMSE'] - df['train_RMSE']
df["RMSE_diff"] = new
#print(df.columns)
display(df[['train_RMSE', 'test_RMSE', 'RMSE_diff', 'sec_conv', '#_epochs', '#filters_conv1', 'filter_size_conv1', 
            '#filters_conv2', 'filter_size_conv2', 'act_func_conv2']].sort_values('RMSE_diff'))
pd.set_option('display.max_rows', 60)

,train_RMSE,test_RMSE,RMSE_diff,sec_conv,#_epochs,#filters_conv1,filter_size_conv1,#filters_conv2,filter_size_conv2,act_func_conv2
62,13.962201,14.687199,0.724997,True,5,128,5,128,5,relu
35,14.271303,15.350747,1.079444,True,5,32,5,64,8,relu
46,13.897146,15.138343,1.241197,True,5,128,5,64,5,relu
44,13.561750,14.825931,1.264180,True,5,128,5,64,2,relu
5,13.781685,15.056663,1.274978,True,5,48,5,16,3,relu
53,13.648573,14.972650,1.324077,True,5,48,5,128,3,relu
14,13.924187,15.271171,1.346984,True,5,128,5,16,5,relu
7,14.427054,15.778987,1.351933,True,5,48,5,16,8,relu
22,13.582807,14.939024,1.356217,True,5,48,5,32,5,relu
43,14.535997,15.907665,1.371668,True,5,64,5,64,8,relu


In [15]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('CNN_log.csv')
# log_df = pd.DataFrame(log)
# # print(len(log_df.columns))
# # print(len(df.columns))
# # display(log_df.columns)
# # display(df.columns)
# log_df.insert(0, 'tuning_group', 2)
# log_df.head()
# df_new = pd.concat([df, log_df], ignore_index = True)
# df_new[df_new['tuning_group'] == 2]

#df_new.to_csv('CNN_log.csv', index = False)
#df['tuning_group'] = 1
#df.insert(0, 'tuning_group', 1)
df["RMSE_diff"] = df['test_RMSE'] - df['train_RMSE']
df = df.sort_values("RMSE_diff").head(20)
df.sort_values("train_RMSE")

,tuning_group,train_MSE,test_MSE,train_RMSE,test_RMSE,train_MAE,test_MAE,train_MAPE,test_MAPE,train_R2,test_R2,sec_conv,perform_validation,num_train_samples,input_1,input_2,#_epochs,#filters_conv1,filter_size_conv1,act_func_conv1,#filters_conv2,filter_size_conv2,act_func_conv2,RMSE_diff
104,2,182.426514,212.404689,13.506536,14.574110,10.114958,10.908070,20866102.0,145.479585,0.895591,0.877000,True,False,17731,30,14,6,64,3,sigmoid,16,5,relu,1.067575
98,2,191.195053,225.890692,13.827330,15.029660,10.439308,11.252300,14013412.0,146.578258,0.890573,0.869191,True,False,17731,30,14,6,48,3,sigmoid,16,5,relu,1.202331
108,2,191.371445,225.584249,13.833707,15.019462,10.307005,11.124750,19191726.0,147.818879,0.890472,0.869368,True,False,17731,30,14,8,32,2,sigmoid,16,5,relu,1.185755
80,2,196.891724,217.193315,14.031811,14.737480,10.598783,11.363609,23782782.0,145.900085,0.887313,0.874227,True,False,17731,30,14,4,48,3,sigmoid,16,5,relu,0.705669
92,2,203.547073,235.432022,14.266993,15.343794,10.807105,11.587382,20303906.0,141.639743,0.883504,0.863665,True,False,17731,30,14,6,32,3,sigmoid,16,5,relu,1.076802
74,2,205.398773,224.556847,14.331740,14.985221,10.873980,11.501239,21958748.0,142.646581,0.882444,0.869963,True,False,17731,30,14,4,32,3,sigmoid,16,5,relu,0.653481
90,2,208.430099,232.652863,14.437108,15.252962,10.974200,11.952559,23291880.0,144.217127,0.880709,0.865275,True,False,17731,30,14,6,32,2,sigmoid,16,5,relu,0.815854
84,2,211.284912,230.856975,14.535643,15.193978,11.088404,11.952584,26114678.0,146.935191,0.879075,0.866315,True,False,17731,30,14,4,64,2,sigmoid,16,5,relu,0.658336
120,2,212.591370,241.210654,14.580513,15.530958,11.269651,12.287003,50776016.0,142.937367,0.878327,0.860319,True,False,17731,30,14,8,64,2,sigmoid,16,5,relu,0.950445
86,2,212.676147,235.933260,14.583421,15.360119,11.023226,11.226655,24636062.0,152.892104,0.878279,0.863375,True,False,17731,30,14,4,64,3,sigmoid,16,5,relu,0.776698


In [14]:
def make_tuning_plot(dataframe):
    plot = alt.Chart(dataframe).mark_line().encode(x = '#_epochs', 
                                                   y = 'RMSE_diff', 
                                                   color = "#filters_conv1", 
                                                   strokeDash = 'act_func_conv1')
    return plot

df_temp = pd.read_csv('CNN_log.csv')
df_temp['RMSE_diff'] = df_temp['test_RMSE'] - df_temp['train_RMSE']
display(df_temp.head(5))

chart = alt.Chart(df_temp[df_temp['tuning_group'] == 1]).mark_point().encode(x = "test_RMSE", 
                                                                             y = 'RMSE_diff', 
                                                                             color = "act_func_conv1", 
                                                                             shape = '#_epochs')
chart

,tuning_group,train_MSE,test_MSE,train_RMSE,test_RMSE,train_MAE,test_MAE,train_MAPE,test_MAPE,train_R2,...,input_1,input_2,#_epochs,#filters_conv1,filter_size_conv1,act_func_conv1,#filters_conv2,filter_size_conv2,act_func_conv2,RMSE_diff
0,1,187.457962,282.402136,13.691529,16.804825,10.799602,13.084684,55712464.0,149.113787,0.893450,...,30,14,5,32,5,relu,64,5,relu,3.113296
1,1,166.968964,251.880678,12.921647,15.870749,10.166595,11.988082,50640868.0,146.791805,0.904438,...,30,14,5,32,5,relu,64,5,relu,2.949102
2,1,235.376343,266.303088,15.341980,16.318796,11.758848,12.331007,36186320.0,138.621394,0.866213,...,30,14,5,32,5,sigmoid,64,5,relu,0.976816
3,1,206.632874,245.249403,14.374730,15.660441,11.081713,11.605846,29879520.0,148.520952,0.881737,...,30,14,5,32,5,sigmoid,64,5,relu,1.285711
4,1,149.568848,253.170068,12.229835,15.911319,9.406377,12.348109,24409952.0,144.298868,0.914986,...,30,14,5,32,5,tanh,64,5,relu,3.681484


SchemaValidationError: 'quantitative' is an invalid value for `type`. Valid values are one of ['nominal', 'ordinal', 'geojson'].

alt.Chart(...)

In [334]:
#Make and train a model using a smaller volume of data (for testing only).
cnn = make_cnn()
history = cnn.fit(train, y_train, 
                  epochs = 10, #number of iterations over the network
                  verbose=0,  #prints out progress during training: options 0, 1, and 2
                  #validation_split = 0.2, #fraction is percentage to leave out
                  #validation_data = None, #can use in place of the above parameter "validation_split"
                 )

In [335]:
history.history.keys()

dict_keys(['loss', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_error', 'r2_score', 'root_mean_squared_error'])

In [337]:
results = cnn.evaluate(test, y_test)
results

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 267.0555 - mean_absolute_error: 13.1149 - mean_absolute_percentage_error: 21.9837 - mean_squared_error: 267.0555 - r2_score: 0.8227 - root_mean_squared_error: 16.3283


[276.0820617675781,
 276.0820617675781,
 16.6157169342041,
 13.015892028808594,
 22.702119827270508,
 0.8401257395744324]

In [338]:
for m in ["mean_squared_error", 
          "root_mean_squared_error", 
          "mean_absolute_error", 
          "mean_absolute_percentage_error", 
          "r2_score"]:
    print(m, ": ", history.history[m][-1])
    
print(history.history["root_mean_squared_error"][:5])
print(history.history["root_mean_squared_error"][-5:])

mean_squared_error :  120.1983642578125
root_mean_squared_error :  10.963501930236816
mean_absolute_error :  8.364891052246094
mean_absolute_percentage_error :  19090606.0
r2_score :  0.9312067627906799
[27.749483108520508, 18.598886489868164, 14.675307273864746, 13.871909141540527, 13.20733642578125]
[12.907819747924805, 12.222270965576172, 11.732117652893066, 11.317305564880371, 10.963501930236816]


In [339]:
y_hat = cnn.predict(test, verbose = 0)

In [340]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_hat)
mse = mean_squared_error(y_test, y_hat)
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
mape = np.mean(np.abs((y_test - y_hat) / y_test)) * 100
r2 = r2_score(y_test, y_hat)

print("MSE: ", mse)
print("RMSE: ", rmse)
print("MAE: ", mae)
print("MAPE: ", mape)
print("R2 Score: ", r2)

MSE:  276.0820318143302
RMSE:  16.61571640990331
MAE:  13.015892562866211
MAPE:  155.50087114852096
R2 Score:  0.8401256799697876


In [341]:
y_hat = cnn.predict(train, verbose = 0)

In [342]:
mae = mean_absolute_error(y_train, y_hat)
mse = mean_squared_error(y_train, y_hat)
rmse = np.sqrt(mean_squared_error(y_train, y_hat))
mape = np.mean(np.abs((y_train - y_hat) / y_train)) * 100
r2 = r2_score(y_train, y_hat)

print("MSE: ", mse)
print("RMSE: ", rmse)
print("MAE: ", mae)
print("MAPE: ", mape)
print("R2 Score: ", r2)

C:\Users\sbmoi\AppData\Local\Temp\ipykernel_48700\1387731807.py:4: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_train - y_hat) / y_train)) * 100


MSE:  118.24085993826348
RMSE:  10.873861316858124
MAE:  8.029988052255494
MAPE:  inf
R2 Score:  0.9323269717559578


# CNN with PyTorch

In [ ]:
import torch

#This module contains stateful NN layers, are object-oriented and automatically learns manages parameters
import torch.nn as nn 

#This module has stateless layers and adds flexibility over those in torch.nn, requires customization
import torch.nn.functional as F

#Used for images - NOT REQ'D
import torchvision

#Image Transformations - NOT REQ'D
import torchvision.transforms as transforms